In [1]:
import numpy as np
import pandas as pd 

In [2]:
import sklearn
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [6]:
train_df = pd.read_csv("C:\\Users\\tanbe\\Desktop\\gb-classification-choose-tutors\\train.csv")
test_df = pd.read_csv("C:\\Users\\tanbe\\Desktop\\gb-classification-choose-tutors\\test.csv")

**Загрузка данных**

Описание датасета:
    
id - идентификатор

age - возраст репетитора

years_of_experience - опыт

lesson_price - стоимость одного урока

qualification - квалификация

physics - физика

chemistry - химия

biology - биология

english - английский

geography - география

history - история

mean_exam_points - средний балл за экзамен

choose - целевая переменная (1 - подходит/0 - нет)

In [7]:
train_df.head()

,Id,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history,mean_exam_points,choose
0,0,35.0,0.0,2150.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,74.0,0
1,1,52.0,2.0,1250.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,57.0,1
2,2,29.0,3.0,1750.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,66.0,0
3,3,33.0,3.0,1050.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,66.0,1
4,4,46.0,3.0,2250.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,73.0,0


**Обработка пропусков**

In [8]:
train_df.isna().sum()

Id                     0
age                    0
years_of_experience    0
lesson_price           0
qualification          0
physics                0
chemistry              0
biology                0
english                0
geography              0
history                0
mean_exam_points       0
choose                 0
dtype: int64

Пропуски отсутствуют

**Так как нам нужен репетитор по математике - нас не интересуют другие предметы, следовательно эти признаки можно удалить**

In [9]:
def drop_column(column_list, train_df, test_df):
    for col in column_list:
        train_df = train_df.drop([col], axis=1)
        test_df = test_df.drop([col], axis=1)


column_list = ['history', 'geography', 'english']   
drop_column(column_list, train_df, test_df)

**Разделим выборку на обучающую и тестовую в соотношении 75/25**

In [10]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(train_df.drop(['Id', 'choose'], axis=1),
                                                                    train_df['choose'], test_size=0.25)

**Преобразовываем pandas dataframe в numpy arrays**

In [11]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [12]:
#Используем евклидову метрику. Реализуем функцию для ее подсчета.

def e_metrics(x1, x2):

    distance = np.sum(np.square(x1 - x2))

    return np.sqrt(distance)

In [13]:
# Используем оптимизированный код KNN c нулём циклов
def knn(x_train, y_train, x_test, k, weights=None):
    
    # расчет расстояний от классифицируемых объектов до
    # объектов обучающей выборки
    distances = np.sqrt(np.sum(np.square(x_test[:, np.newaxis] - x_train), axis=2))
                
    answers = []    
    for distance in distances:
        # Записываем в список значение расстояния и ответа на объекте обучающей выборки
        test_distances = [(dist, label) for dist, label in zip(distance, y_train)]
        # создаем словарь со всеми возможными классами
        classes = {class_item: 0 for class_item in set(y_train)}
        
        # Сортируем список и среди первых k элементов подсчитаем частоту появления разных классов
        for d in sorted(test_distances)[0:k]:
            classes[d[1]] += 1

        # Записываем в список ответов наиболее часто встречающийся класс
        answers.append(sorted(classes, key=classes.get)[-1])
        
    return answers

In [14]:
# Функция для вычисления точности
def accuracy(pred, y):
    return (sum(pred == y) / len(y))

**Методом подбора выяснилось, что лучшая точность получается при k=10**

In [15]:
k = 10

%time y_pred_train = knn(X_train, y_train, X_train, k)
%time y_pred = knn(X_train, y_train, X_test, k)

print(f'Точность алгоритма на трейне при k = {k}: {accuracy(y_pred_train, y_train):.3f}')
print(f'Точность алгоритма на тесте при k = {k}: {accuracy(y_pred, y_test):.3f}')

Wall time: 1min 27s
Wall time: 26.2 s
Точность алгоритма на трейне при k = 10: 0.901
Точность алгоритма на тесте при k = 10: 0.882


In [16]:
X = train_df.drop(['Id', 'choose'], axis=1).to_numpy()
y = train_df['choose'].to_numpy()
test_np = test_df.drop(['Id'], axis=1).to_numpy()

In [17]:
k = 10

%time y_pred_test = knn(X, y, test_np, k)

print(f'Точность алгоритма на трейне при k = {k}: {accuracy(y_pred_test, y):.3f}')

Wall time: 3min 3s
Точность алгоритма на трейне при k = 10: 0.866


In [19]:
submission = pd.DataFrame({'Id': test_df['Id'], 'choose': y_pred_test})
submission.to_csv(f"C:\\Users\\tanbe\\Desktop\\gb-classification-choose-tutors\\sample_submission.csv", index=False, encoding='utf-8')